In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader

In [ ]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(5, 10, 4 )
        self.conv2 = nn.Conv1d(10, 10, 4)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(10 * 13, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x,2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [ ]:
conv_net = ConvNet()
print(conv_net)

In [ ]:
input = torch.randn(128, 5, 32)
out = conv_net(input)
print(input.shape)
print(out.shape)

In [ ]:
criterion = nn.L1Loss()
optimizer = optim.SGD(conv_net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
midi_dataset = MidiDataset("training_data.npy")
dataloader = DataLoader(midi_dataset, batch_size=32, shuffle=True)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[:,:,0:32], data[:,:,32]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = conv_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
for i, data in enumerate(dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    if i == 0:
        print( data.shape)
    inputs, labels = data[5:7,:,0:33], data[5:7,:,32]
    print("labels:", labels)
    print("pred:", conv_net(inputs))
    
    if i == 2:
        break

In [ ]:
#torch.save(conv_net.state_dict(), "20epochs.pth")

In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load("20epochs.pth"))
conv_net.eval()

In [ ]:
midi_test_dataset = MidiDataset("test_data.npy", midi_dataset.mean, midi_dataset.std)
test_dataloader = DataLoader(midi_test_dataset, batch_size=64, shuffle=False)

In [ ]:
for i, data in enumerate(test_dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    print(data.shape)
    inputs, labels = data[1:64,:,0:32], data[1:64,:,32]
    #print(inputs)
    print("labels:", labels)
    print("pred:", conv_net(inputs))
    